In [1]:
from ScicatTool.Filesystem.FSInfo import listdir, list_files, file_size, get_creation_date
from ScicatTool.Filesystem.FSInfoWin import get_ownername
from ScicatTool.Filesystem.ImInfo import load_numpy_from_image, TYPES, SUPPORTED_IMAGE_TYPES, get_uri_from_numpy, get_ext
from ScicatTool.Datasets.APIKeys import SOURCE_FOLDER, SIZE, PID, ATTACHMENT_CAPTION
from ScicatTool.Datasets.Dataset import ScientificMetadataBuilder, AttachmentBuilder
from ScicatTool.Datasets.Consts import PID_PREFIX
from ScicatTool.Datasets.DatasetVirtual import HistoRawDatasetBuilder
from ScicatTool.Datablocks.Datablock import OrigDatablockBuilder
from ScicatTool.Datablocks.APIKeys import DATA_FILE_LIST
from ScicatTool.Proposals.APIKeys import PROPOSAL_ID, LASTNAME
from ScicatTool.Sites.Beamline.Consts import RAW
from ScicatTool.REST.Consts import NA
import ScicatTool.REST.API as API
from pprint import pprint
import argparse
import numpy as np
import csv


PATH = "D:/PowerFolders/Synchroload MgBone/Histologies"
EXTS = [".tif", ".tiff", ".jpg", ".jpeg"]
FILENAME_IGNORE_PATTERN = ["_ref", "_dar"]
class ArgsMockUp():
    token = "04zCAnxYXryYV2kbFQodMBEqyRHGykTKBFCf9imjAYnDL7x5tNiCC0guUW2ahA30"
    filename = "C:/Users/lucaschr/mdlma-scicat-ingestion/_samples_synchroload.csv"
    histoidcol = 14
    ownergroup = "hasylab"
    accessgroups = ["public", "it", "wb", "hasylab", "external"]
    contactemail = "julian.moosmann@desy.de"
    keywords = []
    nattachments = 4
    thumbnailsize = 150
    verbose = 0
    simulation = False
    blankdatablock = False

In [2]:
def get_histoids(args):    
    header = None
    first = True
    samples = {}
    with open(args.filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=';', quotechar='"')
        for row in reader:
            if first:
                header = row
                first = False
            else:
                if row[0] in samples.values():
                    print("Sample duplicate", row[0])
                    exit(0)
                else:
                    samples[row[args.histoidcol]] = row[0]
    return samples


def _api_dataset_ingest(args, dataset_dict, datablock_dict, attachment_dicts):
    failed = {}
    if not datablock_dict[DATA_FILE_LIST] and not blankdatablock:
        failed[dataset_dict[SOURCE_FOLDER]] = "No files found, thus dataset not added to Scicat!"
    else:
        # Add dataset
        resp = API.dataset_ingest(args.token, dataset_dict, args.simulation, args.verbose)
        if resp.status_code != 200:
            failed[dataset_dict["datasetName"]] = resp.text

        # Add dataset files in a OrigDatablock
        resp = API.origdatablock_ingest(args.token, datablock_dict, args.simulation, args.verbose)
        if resp.status_code != 200:
            failed[dataset_dict["datasetName"] + "-OrigDataBlock"] = resp.text

        # Add attachments
        for attachment_dict in attachment_dicts:
            resp = API.dataset_attach(args.token, attachment_dict, PID_PREFIX + dataset_dict[PID], args.simulation, args.verbose)
            if resp.status_code != 200:
                failed[dataset_dict["datasetName"] + "-" + attachment_dict[ATTACHMENT_CAPTION]] = resp.text

    return failed

In [3]:
args = ArgsMockUp()

histoid_sample = get_histoids(args)

filelist = list_files(PATH, EXTS)

images = {}
for hid in histoid_sample.keys():
    images[hid] = []
    
for fn in filelist:
    pos = fn.find("17")
    if pos > -1:
        histo_id = fn[pos:pos+5]
        if histo_id in histoid_sample.keys():
            try:
                int(histo_id)
            except:
                print(histo_id, "-> nan")
                continue
            images[histo_id].append(fn)

In [4]:
failed = {}
for hid, filename_list in images.items():
    if filename_list:        
        smb = ScientificMetadataBuilder()
        mins = {}
        maxs = {}
        
        full_size = 0
        for img in filename_list:
            img_array, img_format, _ = load_numpy_from_image(PATH + "/" + img)
            if img_array is not None:
                full_size += file_size(PATH, img)
                smb.add_value(img_format + " datatypes", img_array.dtype.name) 
                for i in range(len(img_array.shape)):
                    key = img_format + " shape[" + str(i) + "]"
                    if key not in mins.keys():
                        vmin = np.inf
                        vmax = -np.inf
                    else:
                        vmin = mins[key]
                        vmax = maxs[key]
                    mins[key] = min(vmin, img_array.shape[i])
                    maxs[key] = max(vmax, img_array.shape[i])

        for key in mins.keys():
            smb.set_value(key + " min", mins[key])
            smb.set_value(key + " max", maxs[key])
        
        sampleid = histoid_sample[hid]
        
        dataset_name = "HZG/{}/{}/{}".format(hid, sampleid, RAW)
        proposal_dict = {PROPOSAL_ID: NA, LASTNAME: NA}
        
        creation_time = get_creation_date(PATH + "/" + filename_list[0])

        # 1. create dataset
        dataset_dict = HistoRawDatasetBuilder(base_keywords=["histology"]).\
            args(args).\
            proposal_id(proposal_dict[PROPOSAL_ID]).\
            owner(get_ownername(PATH)).\
            source_folder(PATH).\
            is_published(False).\
            size(full_size).\
            principal_investigator(proposal_dict[LASTNAME]).\
            dataset_name(dataset_name).\
            creation_location("HZG").\
            creation_time(creation_time).\
            scientific_metadata(smb.build()).\
            number_of_files(len(filename_list)).\
            sample_id(sampleid).\
            build()

        # 2. create data file lists
        datablock_dict = OrigDatablockBuilder(dataset_dict[SOURCE_FOLDER], win_fs=True).\
            args(args).\
            data_file_list(filename_list).\
            size(dataset_dict[SIZE]).\
            dataset_id(dataset_dict).\
            build()
        
        # 3. create attachments
        attachment_dicts = []
        images_filename_list = [fn for fn in filename_list if TYPES[get_ext(fn)] in SUPPORTED_IMAGE_TYPES]
        select_filename_list = [fn for fn in images_filename_list if not any([pattern in fn for pattern in FILENAME_IGNORE_PATTERN])]
        sorted_filename_list = sorted(select_filename_list)
        len_list = len(sorted_filename_list)
        if args.nattachments > 0 and len_list > 0:
            step = len_list//args.nattachments
            if step == 0:
                step = 1
            for i in range(0, len_list, step):
                full_path = "{}/{}".format(dataset_dict[SOURCE_FOLDER], sorted_filename_list[i])
                img_array, _, _ = load_numpy_from_image(full_path)
                if img_array is None:
                    failed[full_path] = "Failed to add attachment due to file format, or shape, or ..."
                else:
                    image_uri = get_uri_from_numpy(img_array, target_size=(args.thumbnailsize, args.thumbnailsize))
                    ab = AttachmentBuilder().\
                        args(args).\
                        thumbnail(image_uri).\
                        caption(sorted_filename_list[i]).\
                        proposal_id(proposal_dict[PROPOSAL_ID])
                    attachment_dicts += [ab.build()]
        
        # 4. Add all to Scicat
        failed.update(_api_dataset_ingest(args, dataset_dict, datablock_dict, attachment_dicts))

Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


DATASETS JSON INGEST: <Response [200]>
ORIGDATABLOCKS JSON INGEST: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


DATASETS JSON INGEST: <Response [200]>
ORIGDATABLOCKS JSON INGEST: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


...

DATASETS JSON INGEST: <Response [200]>
ORIGDATABLOCKS JSON INGEST: <Response [200]>


OME series: incompatible page shape (5351, 12123, 3); expected (3, 5351, 12123)


DATASETS JSON ATTACH: <Response [200]>


OME series: incompatible page shape (5351, 12123, 3); expected (3, 5351, 12123)
Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


DATASETS JSON INGEST: <Response [200]>
ORIGDATABLOCKS JSON INGEST: <Response [200]>
DATASETS JSON ATTACH: <Response [200]>


In [5]:
pprint(failed)

{'D:/PowerFolders/Synchroload MgBone/Histologies/Mg-10Gd_12w_17019-0005_66R.tif': 'Failed '
                                                                                  'to '
                                                                                  'add '
                                                                                  'attachment '
                                                                                  'due '
                                                                                  'to '
                                                                                  'file '
                                                                                  'format, '
                                                                                  'or '
                                                                                  'shape, '
                                                                                  'or '
       